In [2]:
library(data.table)
library(DescTools)

NameError: name 'library' is not defined

In [1]:
df=read.table("/cerllar/users/tsears/projects/germline-immune/data/icb.pd1.pdl1.tmb.germline.tsv",sep="\t",header=TRUE)

NameError: name 'read' is not defined

In [3]:
unique(df$study)[0:8]

[1] "riaz"                 "miao"                 "vanallen"            
[4] "hugo"                 "rizvi"                "snyder"              
[7] "cristescu_urothelial" "cristescu_tnbc"

In [4]:
head(df)

,FID,PDCD1,CD274,CTLA4,TMB,burden,pheno,study,IID,Age,⋯,C2,C3,C4,C5,C6,C7,C8,C9,C10,response
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
1,SRR5134823,2.3757345,3.416840,2.4568061,45.0,0.5000000,2,riaz,SRR5134823,52,⋯,-0.0346098,0.016462700,-0.01086980,0.049784300,-0.02963000,0.00103440,-0.03465830,-1.65245e-05,0.045567500,1
2,SRR6504839,3.6892992,3.001802,3.2986583,58.0,0.3636364,1,miao,SRR6504839,52,⋯,-0.0142743,0.008448920,0.00377494,-0.001881900,0.00172930,-0.00303252,-0.00641264,-1.35799e-03,0.010186900,0
3,SRR2724351,0.4005379,1.589763,0.2141248,NA,0.5000000,1,vanallen,SRR2724351,61,⋯,-0.0103183,0.005870190,0.00385378,0.001428540,-0.00727709,0.00454425,-0.00213939,3.76886e-03,-0.009795290,0
4,SRR6511550,1.6735564,1.356144,1.4750849,9.0,0.4545455,2,miao,SRR6511550,59,⋯,-0.0124123,0.004504520,-0.00325456,0.001832370,0.00304893,-0.00451186,0.00105441,-5.88897e-03,-0.008792160,1
5,SRR3083852,0.6229304,3.269033,2.4436067,339.0,0.5000000,2,hugo,SRR3083852,82,⋯,-0.0133546,0.000326364,-0.00125437,0.000724204,0.00191811,-0.00434421,0.00199442,-3.44412e-03,0.002580710,1
6,SRR5134900,1.7224660,2.972693,2.9855004,32.5,0.5000000,2,riaz,SRR5134900,61,⋯,-0.0119105,0.004152100,-0.00108714,0.000102251,0.00334026,-0.00452027,-0.00957989,3.85907e-03,-0.000828938,1


In [5]:
df_results = data.frame(study = character(), model = character(), r2=numeric())

for (study in unique(df$study)){

    print(study)
    df_study=df[df$study==study,]
    
    if (nrow(df_study)>10){
    df_study=merge(df_study[c("FID","study","response")],scale(df_study[c("PDCD1","CD274","TMB","CTLA4","LASSO_burden")],center=T), by.x=0,by.y=0)
    
    tryCatch({
    #germline PRS alone
    study_prs<-glm(response ~ LASSO_burden, data = df_study,family = "binomial")
    print(PseudoR2(study_prs))
    df_results=rbind(df_results,data.frame(study=study,model="germline PRS only",r2=PseudoR2(study_prs,which="all")))
        }
    )

    tryCatch({
    #TMB/PDL1/PD1 
    study_prs<-glm(response ~ TMB, data = df_study,family = "binomial")
    print(PseudoR2(study_prs))
    df_results=rbind(df_results,data.frame(study=study,model="TMB",r2=PseudoR2(study_prs,which="all")))
                }, error=function(e){print("ERROR")})
        
    tryCatch({
    #TMB/PDL1/PD1 
    study_prs<-glm(response ~ TMB + PDCD1 + CD274 + CTLA4, data = df_study,family = "binomial")
    print(PseudoR2(study_prs))
    df_results=rbind(df_results,data.frame(study=study,model="TMB/PD1/PDL1/CTLA4",r2=PseudoR2(study_prs,which="all")))
                }, error=function(e){print("ERROR")})
        
    tryCatch({
    #TMB/PDL1/PD1 + germline PRS
    study_prs<-glm(response ~ TMB + PDCD1 + CD274 +CTLA4 + LASSO_burden, data = df_study,family = "binomial")
    print(PseudoR2(study_prs))
    df_results=rbind(df_results,data.frame(study=study,model="TMB/PD1/PDL1/CTLA4+germline",r2=PseudoR2(study_prs,which="all")))
                }, error=function(e){print("ERROR")})
    
}

}

[1] "riaz"
 McFadden 
0.2378576 
   McFadden 
0.005865639 
  McFadden 
0.07960989 
 McFadden 
0.3638099 
[1] "miao"
  McFadden 
0.02479299 
 McFadden 
0.0177138 
  McFadden 
0.08974435 
  McFadden 
0.09252941 
[1] "vanallen"
 McFadden 
0.1757838 
  McFadden 
0.06211065 
 McFadden 
0.1815904 
 McFadden 
0.4807361 
[1] "hugo"
 McFadden 
0.2543428 
  McFadden 
0.06455895 
  McFadden 
0.07323529 
 McFadden 
0.2247198 
[1] "rizvi"
  McFadden 
0.04966792 
[1] "ERROR"
[1] "ERROR"
[1] "ERROR"
[1] "snyder"
 McFadden 
0.2069816 
[1] "ERROR"
[1] "ERROR"
[1] "ERROR"
[1] "cristescu_urothelial"
    McFadden 
5.400543e-05 
[1] "ERROR"
[1] "ERROR"
[1] "ERROR"
[1] "cristescu_tnbc"
  McFadden 
0.08533651 
[1] "ERROR"
[1] "ERROR"
[1] "ERROR"
[1] "cristescu_er_her_breast"
[1] "cristescu_sclc"
[1] "cristescu_colon_rectaladeno"
[1] "cristescu_hnscc"
   McFadden 
0.003645155 
[1] "ERROR"
[1] "ERROR"
[1] "ERROR"
[1] "cristescu_melanoma"
    McFadden 
3.448611e-06 
[1] "ERROR"
[1] "ERROR"
[1] "ERROR"
[1] "liu"

In [6]:
write.csv(df_results,"/cellar/users/tsears/projects/germline-immune/data/r2.results.csv")